This code varies the design parameters for the single qubit-resonator example and varies them according to a gaussian distribution with a set percentage of std. deviation. The later part of the codes then plots the results.

# Basic packages

In [1]:
%load_ext autoreload
%autoreload 2
import time

In [2]:
import json
import design as d
import names as n

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)

from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys
close_ansys()

In [3]:
# for this example to store the data in the right place
import os
os.makedirs(os.path.dirname("out/"), exist_ok=True)

## Design assembly

In [4]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um")
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

n.add_design_variables_to_design(design, initial_design_variables)

In [5]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, group=n.GROUP_1)
    d.add_transmon_plus_resonator(design, group=n.GROUP_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, group=n.GROUP_1)
    d.add_chargeline(design, group=n.GROUP_2)

    gui.rebuild()
    gui.autoscale()


render_qiskit_metal_design(design, gui)

position: [ 0.       -2.721745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
position: [0.       4.461745] : direction:[-0. -1.]. position: [0.       3.068255] : direction:[0. 1.].
position: [ 0.       -4.461745] : direction:[0. 1.]. position: [ 0.       -3.078255] : direction:[ 0. -1.].
position: [-4.37 -2.  ] : direction:[ 1. -0.]. position: [-2.7 -1.8] : direction:[-1.  0.].
position: [-4.37  2.  ] : direction:[ 1. -0.]. position: [-2.7  1.8] : direction:[-1.  0.].
position: [ 0.       -2.721745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
position: [0.       4.461745] : direction:[-0. -1.]. position: [0.       3.068255] : direction:[0. 1.].
position: [ 0.       -4.461745] : direction:[0. 1.]. position: [ 0.       -3.078255] : direction:[ 0. -1.].
position: [-4.37 -2.  ] : direction:[ 1. -0.]. position: [-2.7 -1.8] : direction:[-1.  0.].
position: [-4.37  2.  ] : direction:[ 1. -0.]. position: [-2.7  1.8] : direction:[-1.  0

## MiniStudy

In [6]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

In [7]:
def CoupledLineTee_mesh_names(comp_names):
    all_names_to_mesh = [f"prime_cpw_{comp_names}", f"second_cpw_{comp_names}"]
    return all_names_to_mesh

In [8]:
MINI_STUDY_GROUP = n.GROUP_1
MINI_STUDY = ms.get_mini_study_qb_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
)

# Definition of randomization in design parameters

In [9]:
import numpy as np
from copy import deepcopy
from qdesignoptimizer.utils.utils import get_value_and_unit

percentage_variation = 10
no_optimization_runs = 5

randomized_design_variables = deepcopy(initial_design_variables)

# Optimization runs

In [10]:
start_time = None
end_time = None
for optimization_run in range(no_optimization_runs):
    if optimization_run == 0:
        start_time = time.strftime("%Y%m%d-%H%M%S")

    for opt_target in opt_targets:
        # print(opt_target.design_var)
        # print(randomized_design_variables[opt_target.design_var])
        value, unit = get_value_and_unit(initial_design_variables[opt_target.design_var])
        randomized_design_variables[opt_target.design_var] = f"{np.random.normal(value, value*percentage_variation/100)}{unit}"

    n.add_design_variables_to_design(design, randomized_design_variables)


    design_analysis_state = DesignAnalysisState(
        design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
    )
    design_analysis = DesignAnalysis(
        design_analysis_state,
        mini_study=MINI_STUDY,
        opt_targets=opt_targets,
        save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
        update_design_variables=False,
        plot_settings=ps.PLOT_SETTINGS,
        meshing_map=[
            MeshingMap(component_class=CoupledLineTee, mesh_names=CoupledLineTee_mesh_names)
        ],
    )

    group_runs = 2  # 10
    group_passes = 2  # 6
    delta_f = 0.001
    for i in range(group_runs):
        design_analysis.update_nbr_passes(group_passes)
        design_analysis.update_delta_f(delta_f)
        design_analysis.optimize_target({}, {})
        design_analysis.screenshot(gui=gui, run=i)

    if optimization_run == no_optimization_runs-1:
        end_time = time.strftime("%Y%m%d-%H%M%S")

[INFO|2025-03-05 11:35:07]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 11:35AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:35AM [load_ansys_project]: 	Opened Ansys App
INFO 11:35AM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 11:35AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/harshva/Documents/Ansoft/
	Project:   Project179
INFO 11:35AM [connect_design]: No active design found (or error getting active design).
INFO 11:35AM [connect]: 	 Connected to project "Project179". No design detected
11:35AM 15s WARNING [activate_ansys_design]: The design_name=get_mini_study_qb_res was not in active project.  Designs in ac

Initializing new desktop!
PyAEDT INFO: StdOut is enabled
PyAEDT INFO: Log on file is enabled
PyAEDT INFO: Log on Desktop Message Manager is enabled
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\AnsysEM21.2\Win64.
PyAEDT INFO: Launching AEDT with COM plugin using PythonNET.
PyAEDT INFO: AEDT 2021.2 Started with process ID 24872.
PyAEDT INFO: pyaedt v0.6.94
PyAEDT INFO: Python version 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:07:43) [MSC v.1942 64 bit (AMD64)]
PyAEDT INFO: AEDT 2021.2.0 Build Date 2021-06-02 22:21:55
PyAEDT INFO: No project is defined. Project Project179 exists and has been read.
PyAEDT INFO: Active Design set to get_mini_study_qb_res
PyAEDT INFO: Aedt Objects correctly read
position: [ 0.       -2.721745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
position: [0.     

INFO 11:35AM [analyze]: Analyzing setup Setup


Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 4.12026775165452e-23
        energy_elec_substrate = 3.71808185173427e-23
        EPR of substrate = 90.2%

        energy_mag    = 5.03164263578725e-25
        energy_mag % of energy_elec_all  = 1.2%
        

Variation 0  [1/1]

  Mode 0 at 6.07 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               98.8%   2.06e-23 2.516e-25

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 96.89%
	jj_name_qub

WARNING 11:36AM [__init__]: <p>Error: <class 'IndexError'></p>


...

INFO 11:43AM [analyze]: Analyzing setup Setup


Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 1.27328136615681e-23
        energy_elec_substrate = 1.16411258613253e-23
        EPR of substrate = 91.4%

        energy_mag    = 1.27330893219803e-23
        energy_mag % of energy_elec_all  = 100.0%
        

Variation 0  [1/1]

  Mode 0 at 6.08 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
                0.0%  1.148e-23 1.148e-23

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 99.97%
	jj_name_q

WARNING 11:43AM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project179\get_mini_study_qb_res\2025-03-05 11-43-45.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization
Pm_norm=
modes
0     1.044559
1    10.820716
dtype: float64

Pm_norm idx =
   jj_name_qubit_1
0            False
1            False
*** P (participation matrix, not normlz.)
   jj_name_qubit_1
0     3.719039e-05
1     9.561688e-07

*** S (sign-bit matrix)
   s_jj_name_qubit_1
0                  1
1                 -1
*** P (participation matrix, normalized.)
   3.7e-05
   9.6e-07

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
  3.92e-09 2.23e-10
  2.23e-10 3.16e-12

*** Chi matrix ND (MHz) 
        -0       -0
        -0       -0

*** Frequencies O1 PT (MHz)
0    6084.237277
1    6725.299640
dtype: float64

*** Frequencies ND (MHz)
0    6084.237277
1    6725.299640

[INFO|2025-03-05 11:43:51]: freq_ND_results
{
    "0": {
        "0": 6084.237276849999,
        "1": 6725.299640030001
    }
}


In [11]:
print(start_time)
print(end_time)

20250305-113507
20250305-114351


# Plotting results

In [21]:
from analysis_functionality import load_data_by_date, plot_optimization_results

files = load_data_by_date('out', CHIP_NAME, start_time, end_time)
plot_optimization_results(files, plot_variance= False)

In [18]:
files

['out\\multi_transmon_chip_20250305-113507.npy',
 'out\\multi_transmon_chip_20250305-113657.npy',
 'out\\multi_transmon_chip_20250305-113840.npy',
 'out\\multi_transmon_chip_20250305-114026.npy',
 'out\\multi_transmon_chip_20250305-114204.npy']

In [ ]:
close_ansys()